In [1]:
import numpy as np
import cv2
import os

## Read image and binarize

In [11]:
img = cv2.imread('lena.bmp', 0)
_, img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
height, width = img.shape[:2]

In [37]:
kernel = np.array([[0,255,255,255,0],
                   [255,255,255,255,255],
                   [255,255,255,255,255],
                   [255,255,255,255,255],
                   [0,255,255,255,0]], dtype='uint8')

## (a) Dilation

In [38]:
def dilation(img, kernel, padding=0):
    img_h, img_w = img.shape[0:2]
    imap = np.zeros((img_h+2*padding, img_w+2*padding), dtype='uint8')
    imap_h, imap_w = imap.shape[0:2]
    
    for i in range(padding, imap_h-padding):
        for j in range(padding, imap_w-padding):
            if img[i-padding,j-padding] == 255:
                imap[i-padding:i+padding+1, j-padding:j+padding+1] |= kernel
                
    dilated = imap[padding:imap_h-padding, padding:imap_w-padding]
    return dilated
    

dilated = dilation(img, kernel, padding=2)
cv2.imshow("Dilated", dilated)
cv2.waitKey(0)
cv2.destroyAllWindows()

## (b) Erosion

In [75]:
def erosion(img, kernel, padding=0):
    img_comp = cv2.bitwise_not(img)
    img_h, img_w = img.shape[0:2]
    imap = np.zeros((img_h+2*padding, img_w+2*padding), dtype='uint8')
    imap_h, imap_w = imap.shape[0:2]
    
    ## Flip kernel 
    _kernel = np.flip(np.flip(kernel, axis=1), axis=0)
    
    for i in range(padding, imap_h-padding):
        for j in range(padding, imap_w-padding):
            if img_comp[i-padding,j-padding] == 255:
                imap[i-padding:i+padding+1, j-padding:j+padding+1] |= _kernel
                
    erosioned = imap[padding:imap_h-padding, padding:imap_w-padding]
    erosioned = cv2.bitwise_not(erosioned)
    
    return erosioned

erosioned = erosion(img, kernel, padding=2)
cv2.imshow("Erosioned", erosioned)
cv2.waitKey(0)
cv2.destroyAllWindows()

## (c) Opening

In [51]:
def opening(img, kernel, padding=0):
    erosioned = erosion(img, kernel, padding=padding)
    opened = dilation(erosioned, kernel, padding=padding)
    
    return opened

opened = opening(img, kernel, padding=2)
cv2.imshow("Opened", opened)
cv2.waitKey(0)
cv2.destroyAllWindows()

## (d) Closing

In [60]:
def closing(img, kernel, padding=0):
    dilated = dilation(img, kernel, padding)
    closed = erosion(dilated, kernel, padding)
    
    return closed

closed = closing(img, kernel, padding=2)
cv2.imshow("Closed", closed)
cv2.waitKey(0)
cv2.destroyAllWindows()

## (e) Hit-and-miss transform

In [77]:
kernel_1 = np.array([[0  ,0  ,0  ],
                     [255,255,0  ],
                     [0  ,255,0  ]], dtype='uint8')

kernel_2 = np.array([[0  ,255,255],
                     [0  ,0  ,255],
                     [0  ,0  ,0  ]], dtype='uint8')

In [80]:
def hit_and_miss(img, kernel_1, kernel_2, padding=0):
    img_comp = cv2.bitwise_not(img)
    hit = erosion(img, kernel_1, padding=padding)
    miss = erosion(img_comp, kernel_2, padding=padding)
    
    cv2.imshow("hit", hit)
    cv2.imshow("miss", miss)

    
    ham = cv2.bitwise_and(hit,miss)
    
    return ham

ham = hit_and_miss(img, kernel_1, kernel_2, padding=1)
cv2.imshow("Hit and miss", ham)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [45]:
 x = [[11, 13, 12, 11],
     [24, 23, 22, 21],
     [34, 33, 32, 31]]
    
x = np.flip(x, 1)
np.flip(x, 0)

array([[31, 32, 33, 34],
       [21, 22, 23, 24],
       [11, 12, 13, 11]])